In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

tf.random.set_seed(777)  # for reproducibility

print(tf.__version__)

2.3.0


# Lab 9 - 1 따라하기
## Data Handle 
* data load 
* pre process

In [2]:
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

# make as dataset format
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
dataset = dataset.batch(len(x_data))
dataset

<BatchDataset shapes: ((None, 2), (None, 1)), types: (tf.int32, tf.int32)>

In [3]:
def preprocess_fn(feature, label):
    feature = tf.cast(feature, dtype=tf.float32)
    label = tf.cast(label, dtype=tf.float32)
    return feature, label

## 모델 설정
* first layer will have logistic regression format
* second layer will have another logistic regression format

In [28]:
W1 = tf.Variable(tf.random.uniform(shape=(2,2), minval = -1, maxval = 1), name = 'Weight1')
B1 = tf.Variable(tf.random.uniform(shape=(1,2), minval = -1, maxval = 1), name = 'Bias1')
W2 = tf.Variable(tf.random.uniform(shape=(2,1), minval = -1, maxval = 1), name = 'Weight2')
B2 = tf.Variable(tf.random.uniform(shape=(1,1), minval = -1, maxval = 1), name = 'Bias2')

variables = [W1, B1, W2, B2]

def hypothesis_fn(feature):
    y1 = tf.matmul(feature, W1) + B1
    layer1 = tf.sigmoid(y1)
    y2 = tf.matmul(layer1, W2) + B2
    layer2 = tf.sigmoid(y2)
    return layer2

## loss
* Let's just use binary classification entropy

In [29]:
def loss_fn(predicted, truth):
    bce = tf.keras.losses.BinaryCrossentropy()
    loss = bce(truth, predicted)
    return loss

## training
* I'll use SGD (actually I don't know the difference between the adam, let's figure out later)
* learning rate will be decreased while training

In [30]:
log_path = './9-3-xor'
writer = tf.summary.create_file_writer(log_path)

In [31]:
flag_usingLRDecay = True
def learning_schedule(flag_usingLRDecay, start_lr = 0.1, decay_steps = 1000, decay_rate = 0.9):
    if flag_usingLRDecay:
        learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=start_lr, decay_steps = decay_steps, decay_rate = decay_rate, 
                                                                       staircase=False, name=None)
    else:
        learning_rate = start_lr
    return learning_rate

learning_rate = learning_schedule(flag_usingLRDecay)

In [32]:
def grad_fn(feature, label):
    with tf.GradientTape() as tape:
        # get loss
        loss = loss_fn(hypothesis_fn(feature), label)
    grads = tape.gradient(loss, variables)
    return grads

In [37]:
from tqdm.notebook import tqdm

def training_fn(dataset, epoch = 15000, verbose = 1000):
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    for step in tqdm(range(epoch)):
    # get batch
        for feature, label in dataset:
            feature, label = preprocess_fn(feature, label)
            # grad
            grads = grad_fn(feature, label)
            # update values
            optimizer.apply_gradients(zip(grads, variables))
            with writer.as_default():
                tf.summary.histogram('weight1', W1, step = step)
                tf.summary.histogram('bias1', B1, step = step)
                tf.summary.histogram('weight2', W2, step = step)
                tf.summary.histogram('bias2', B2, step = step)
        with writer.as_default():
            tf.summary.scalar('learning_rate', optimizer._decayed_lr(tf.float32), step = step)
            tf.summary.scalar('loss', loss_fn(hypothesis_fn(feature), label), step = step)
        if step % verbose == 0 :
            print(f'step-{step}\tloss-{loss_fn(hypothesis_fn(feature), label)}\tlearning rate-{optimizer._decayed_lr(tf.float32)}')

In [38]:
epoch = 10000
training_fn(dataset, epoch)

  0%|          | 0/10000 [00:00<?, ?it/s]

step-0	loss-0.7783117294311523	learning rate-0.09998946636915207
step-1000	loss-0.6929596662521362	learning rate-0.0899905189871788
step-2000	loss-0.6917161345481873	learning rate-0.0809914693236351
step-3000	loss-0.6857339143753052	learning rate-0.07289231568574905
step-4000	loss-0.6537238359451294	learning rate-0.06560308486223221
step-5000	loss-0.5666872262954712	learning rate-0.05904277041554451
step-6000	loss-0.47874003648757935	learning rate-0.05313849449157715
step-7000	loss-0.43033552169799805	learning rate-0.047824643552303314
step-8000	loss-0.4059346318244934	learning rate-0.043042175471782684
step-9000	loss-0.3924615979194641	learning rate-0.038737956434488297


## check

In [16]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

In [24]:
print(f'variables - {variables}')
for feature, label in dataset:
    feature, label = preprocess_fn(feature, label)
    print(f'result of trained {accuracy_fn(hypothesis_fn(feature), label)}')

variables - [<tf.Variable 'Weight1:0' shape=(2, 2) dtype=float32, numpy=
array([[-6.269986, -3.62093 ],
       [-6.314083, -3.624776]], dtype=float32)>, <tf.Variable 'Bias1:0' shape=(1, 2) dtype=float32, numpy=array([[2.2659688, 5.2387614]], dtype=float32)>, <tf.Variable 'Weight2:0' shape=(2, 1) dtype=float32, numpy=
array([[-8.256178 ],
       [ 7.2969365]], dtype=float32)>, <tf.Variable 'Bias2:0' shape=(1, 1) dtype=float32, numpy=array([[-3.081015]], dtype=float32)>]
result of trained 1.0


## Save Model to use Later
* 특별한 save function을 설정하지 않았기 때문에, 간단히 weight 만 저장하도록 하자

checkpoint 를 쓰라고 하는데, 일단은 패스